In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# Import data
import numpy as np
import pandas as pd

data = pd.read_csv('/content/gdrive/My Drive/NLP/stance-data.csv')
data.columns

Index(['post_text', 'topic', 'author', 'label', 'id', 'word_count',
       'words_pronom', 'words_per_sen', 'words_over_6', 'pos_emo', 'neg_emo',
       'count_noun', 'count_verb', 'count_adj'],
      dtype='object')

In [0]:
# Select topic and shuffle data
topic = "abortion"
selected = data[data.topic==topic]
selected = np.random.permutation(selected)
#set(selected[:,1])

In [0]:
# Import functions
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, ComplementNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV

# 2. Method 1 - Ngrams (no idf)

In [12]:
# Show that ngram_range don't need to be params because of feature selection 
# cv = CountVectorizer(stop_words='english', ngram_range=(1,4))
# print(np.shape(cv.fit_transform(selected[:,0])))
# cv = CountVectorizer(stop_words='english', ngram_range=(1,2))
# print(np.shape(cv.fit_transform(selected[:,0])))
# cv = CountVectorizer(stop_words='english', ngram_range=(3,4))
# print(np.shape(cv.fit_transform(selected[:,0])))

(1708, 237620)
(1708, 74275)
(1708, 163345)


In [16]:
np.shape(y)

(1708,)

In [0]:
# Set label as y
y = selected[:,3]

# Extract features
X = CountVectorizer(stop_words='english', ngram_range=(1,4))
X = X.fit_transform(selected[:,0])

In [7]:
# Build pipeline with MultinomialNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', MultinomialNB()),
])
params = {
    'exf__k':(20,50,100,500,1000,2000),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.6077309598532009
clf__alpha: 1
exf__k: 500


In [8]:
# Build pipeline with BernoulliNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', BernoulliNB()),
])
params = {
    'exf__k':(20,50,100,500,1000),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.6135789130695752
clf__alpha: 0.5
exf__k: 50


In [10]:
# Build pipeline with ComplementNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', ComplementNB()),
])
params = {
    'exf__k':(20,50,100,500),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.6141877175833033
clf__alpha: 1
exf__k: 100


In [0]:
# Build pipeline with SVC as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', SVC()),
])
params = [  
    {'exf__k':[20,50,100,500],'clf__kernel':['rbf'],'clf__gamma': [1e-3, 1e-4],
     'clf__C': [0.1,1, 10, 100]},
    {'exf__k':[20,50,100,500],'clf__kernel': ['linear'], 'clf__C': [0.1,1, 10, 100]},
]
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)

In [25]:
print(gs.best_score_, gs.best_params_)

0.6071375898201026 {'clf__C': 100, 'clf__gamma': 0.001, 'clf__kernel': 'rbf', 'exf__k': 100}


## 找到abortion最好组合(Ngram)：0.614
0.6141877175833033
clf__alpha: 1
exf__k: 100

ComplementNB


## Gay rights

In [0]:
# Select topic and shuffle data
topic2 = "gay rights"
selected2 = data[data.topic==topic2]
selected2 = np.random.permutation(selected2)
#set(selected2[:,1])
# Set label as y
y = selected2[:,3]

# Extract features
X = CountVectorizer(stop_words='english', ngram_range=(1,4))
X = X.fit_transform(selected2[:,0])

In [30]:
# Build pipeline with MultinomialNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', MultinomialNB()),
])
params = {
    'exf__k':(20,50,100,500,1000,2000),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.6210507819363908
clf__alpha: 1
exf__k: 20


In [31]:
# Build pipeline with BernoulliNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', BernoulliNB()),
])
params = {
    'exf__k':(20,50,100,500,1000),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.6166119243610018
clf__alpha: 1
exf__k: 1000


In [32]:
# Build pipeline with ComplementNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', ComplementNB()),
])
params = {
    'exf__k':(20,50,100,500),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.5791249341064839
clf__alpha: 1
exf__k: 500


In [0]:
# Build pipeline with SVC as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', SVC()),
])
params = [  
    {'exf__k':[20,50,100,500],'clf__kernel':['rbf'],'clf__gamma': [1e-3, 1e-4],
     'clf__C': [0.1,1, 10, 100]},
    {'exf__k':[20,50,100,500],'clf__kernel': ['linear'], 'clf__C': [0.1,1, 10, 100]},
]
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)

In [34]:
print(gs.best_score_, gs.best_params_)

0.6453401457091494 {'clf__C': 100, 'clf__gamma': 0.0001, 'clf__kernel': 'rbf', 'exf__k': 20}


# 3. Method 2-Other features (with idf?)

In [0]:
# Extract features
cv = CountVectorizer(stop_words='english', ngram_range=(1,4))
test_cv = cv.fit_transform(shuffled_test[:,0])


idf???????

from scipy.sparse import hstack
test_X = hstack((test_cv, shuffled_test[:,3:6].astype(float)))
test_y = shuffled_test[:,1]